## Uploading a dataset

In [1]:
from pubweb import DataPortal
portal = DataPortal()

To upload a dataset to the portal, you need to specify:

  1. The project which it will be uploaded to
  2. The process which defines the type of data being uploaded
  3. The name and description of the dataset
  4. The path to the folder which contains the files to upload
  5. Optionally, a filtered list of files which should be uploaded (default is to upload all files in a folder)

First, get the project where the dataset will be uploaded

In [2]:
project = portal.get_project_by_name("Test Project")
print(f"Name: {project.name}")
print(f"ID: {project.id}")

Name: Test Project
ID: 9a31492a-e679-43ce-9f06-d84213c8f7f7


Next, select the process which will be used for ingesting the data

In [3]:
# Get a list of all available ingest processes
ingest_processes = portal.list_processes(ingest=True)
print(f"There are {len(ingest_processes):,} different ingest processes available")
# Uncomment the line below to print a list of all available ingest processes
# print(ingest_processes.description())

# Select the process used to ingest paired DNA sequencing data
process = ingest_processes.get_by_name("Paired DNAseq (FASTQ)")
print("\nSelected process:")
print(str(process))

# To do the above in a single step, simply run:
# process = portal.get_process_by_name("Paired DNAseq (FASTQ)", ingest=True)

# When uploading the dataset (below), the process used for ingest can be specified
# either with the process object which was obtained above, or simply with the name
# of that process (or its ID).

There are 12 different ingest processes available

Selected process:
Name: Paired DNAseq (FASTQ)
Id: paired_dnaseq
Description: FASTQ files generated from paired-end sequencing of DNA libraries


If you want to upload just a subset of files from a folder, you just need to make a list of the filenames (paths relative to the upload directory) which you want to upload. This overrides the default behavior of uploading everything from the upload directory.

We've included two helper functions to get a list of files in the specified directory and filter them.

You can also manually create the list of files (using those relative paths)

In [4]:
from pubweb.file_utils import get_files_in_directory, filter_files_by_pattern

directory_to_upload = '/tmp'

files = get_files_in_directory(directory_to_upload)
files_to_upload = filter_files_by_pattern(files, '*.fastq.gz')
files_to_upload

['test.R1.fastq.gz', 'test.R2.fastq.gz']

Fill in information on your new dataset in the `name` and `description` variables and then run to check the files and upload new data

In [5]:
# Upload the data and return the new dataset object
uploaded_dataset = project.upload_dataset(
    name = 'Test dataset',
    description = '',
    upload_folder = '/tmp',
    files = files_to_upload,
    process = "Paired DNAseq (FASTQ)"
)

print(str(uploaded_dataset))

Uploading file test.R1.fastq.gz (180.76 KB) | 100.0%|█████████████████████████ | 669kB/s
Uploading file test.R2.fastq.gz (180.76 KB) | 100.0%|█████████████████████████ | 1.30MB/s


Name: Test dataset
Id: beccbeea-5ad4-4a76-9c63-08d5d9b63f7b
Description: 
Status: COMPLETED


That was an example of what a successful upload looks like.

Next, let's take a look at one of the most common issues that you might run into. Namely, if you try to upload files which don't match the pattern expected, it will raise an error:

In [7]:
import json
ingest_10X = portal.get_process_by_name("Single-cell sequencing data (10X)", ingest=True)
print("File pattern expected for 10X data (note the sample index number and lane ID):")
print(json.dumps(ingest_10X.file_mapping_rules, indent=3))

# Try to upload the data (which will cause an error)
project.upload_dataset(
    name = 'Test dataset',
    description = '',
    upload_folder = '/tmp',
    files = files_to_upload,
    process = "Single-cell sequencing data (10X)"
)


File pattern expected for 10X data (note the sample index number and lane ID):
[
   {
      "glob": "*_S*_L???_{I,R}{1,2}_001.fastq.gz",
      "min": null,
      "max": null,
      "description": "Paired FASTQ (Illumina Format)",
      "isSample": null,
      "sampleMatchingPattern": "(?P<sampleName>\\S*)_S(?P<sampleIndex>\\S*)_L(?P<lane>\\S*)_(?P<read_index>I|R)(?P<read>1|2)_001\\.fastq\\.gz"
   }
]


ValueError: Files do not match dataset type. Expected file type requirements: 
Paired FASTQ (Illumina Format) *_S*_L???_{I,R}{1,2}_001.fastq.gz